In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
proxy = "http://172.16.2.251:3128"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ajays\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ajays\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def article_scrape(url):
    L = ['', '', '', '', '']
    columns = ["Authors", "Publication date", "Description"]
    r = requests.get(url, proxies = { "https" : proxy})
    soup = BeautifulSoup(r.content, "html.parser")
    
    L[0] = soup.find(name = 'div' , attrs = {"id" : "gsc_oci_title"}).text                                           
    for i in range(len(soup.find_all(name = 'div' , attrs = {"class" : "gsc_oci_field"}))):
        for j in range(len(columns)):
            if (soup.find_all(name = 'div' , attrs = {"class" : "gsc_oci_field"})[i].text == columns[j]):
                L[j+1] = soup.find_all(name = 'div' , attrs = {"class" : "gsc_oci_value"})[i].text

        if (soup.find_all(name = 'div' , attrs = {"class" : "gsc_oci_field"})[i].text == "Total citations"):
            L[4] = soup.find_all(name = 'div' , attrs = {"class" : "gsc_oci_value"})[i].find(name = 'div' , attrs = {"style" : "margin-bottom:1em"}).text.split("by ")[1]
        
    return L


In [3]:
def profile_finder(List):
    i = 0
    L = []
    search_failed = []
    for link in List:
        r = requests.get(link, proxies = { "https" : proxy}) # method used is GET
        s = BeautifulSoup(r.content, "html.parser")
        if (s.find(name = 'div' , attrs = {"class" : "gs_ai_t"}) == None):
            print("Could not search :", prof_ucl[i])
            search_failed.append(prof_ucl[i])
        else:
            L.append("https://scholar.google.com" + s.find(name = 'div' , attrs = {"class" : "gs_ai_t"}).find(name = 'h3' , attrs = {"class" : "gs_ai_name"}).a["href"])
        i += 1
    return L, search_failed

In [4]:
def profile_scraper(url):
    L = []
    articles_highc_link = []      
    articles_recent_link = []
    articles_highc = []      
    articles_recent = []
    r = requests.get(url, proxies = { "https" : proxy})
    soup = BeautifulSoup(r.content, "html.parser")    
    L.append(soup.find(name = 'div' , attrs = {"id" : "gsc_prf_in"}).text)
    L.append([soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[0].text, soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[1].text])
    L.append([soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[2].text, soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[3].text])
    L.append([soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[4].text, soup.find_all(name = 'td' , attrs = {"class" : "gsc_rsb_std"})[5].text])

    for i in range(10):
        articles_highc_link.append("https://scholar.google.com" + soup.find_all(name = 'tr' , attrs = {"class" : "gsc_a_tr"})[i].a["href"])
    
    r = requests.get(url + "&view_op=list_works&sortby=pubdate", proxies = { "https" : proxy})
    soup = BeautifulSoup(r.content, "html.parser")    
    for i in range(5):
        articles_recent_link.append("https://scholar.google.com" + soup.find_all(name = 'tr' , attrs = {"class" : "gsc_a_tr"})[i].a["href"])

    for i in range(len(articles_highc_link)):
        articles_highc.append(article_scrape(articles_highc_link[i]))
    L.append(articles_highc)

    for i in range(len(articles_recent_link)):
        articles_recent.append(article_scrape(articles_recent_link[i]))
    L.append(articles_recent)


    return L
    

In [5]:
def get_prof_names():                 # Webscraping the UCL Quantum site's department place
    r = requests.get("https://www.ucl.ac.uk/quantum/people", proxies = { "https" : proxy}) # method used is GET
    s = BeautifulSoup(r.content, "html.parser")
    pp = s.find_all("a")
    L = []
    for i in range(len(pp)):
        if (i>34 and i<73 and pp[i].text != ""):
            if (pp[i].text == "Prof\xa0Marzena Szymanska"):
                L.append("Prof Marzena Szymanska")
            else:
                L.append(pp[i].text)
    return L

In [6]:
def searchable_links(prof_name):
    L = []
    for i in prof_name:
        string = ""
        i = i.replace("(","%28")
        i = i.replace(")","%29")
        for k in range(1, len(i.split(" "))):
            if(k == len(i.split(" "))-1):
                string += i.split(" ")[k]
            else:
                string += i.split(" ")[k] + "+"
        L.append("https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=" + string + "&btnG=")

    return L

In [7]:
def summarize(text):
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text)
    
    # Creating a frequency table to keep the 
    # score of each word
    
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
    
    # Creating a dictionary to keep the score
    # of each sentence
    sentences = sent_tokenize(text)
    sentenceValue = dict()
    
    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
    
    
    
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]
    
    # Average value of a sentence from the original text
    
    average = int(sumValues / len(sentenceValue))
    
    # Storing sentences into our summary.
    summary = ''
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
            summary += " " + sentence
    return summary

In [40]:
def summarize_abstracts(A):
    L = A
    for i in range(len(L)):
        for k in range(10):
            if L[i][4][k][3] != "" and summarize(L[i][4][k][3]) != "":
                L[i][4][k][3] = summarize(L[i][4][k][3])
                
        for k in range(5):
            if L[i][5][k][3] != "" and summarize(L[i][4][k][3]) != "":
                L[i][5][k][3] = summarize(L[i][5][k][3])

    return L

In [9]:
prof_ucl = get_prof_names()
links = searchable_links(prof_ucl)
final_prof_site = profile_finder(links)[0]

Could not search : Dr Mark Buitelaar
Could not search : Prof Peter Coveney
Could not search : Prof Carla Figueira De Morisson Faria
Could not search : Prof Tania Monteiro
Could not search : Prof Sir Michael Pepper FRS
Could not search : Prof Massimiliamo (Massi) Pontil
Could not search : Prof Andrew Pontzen
Could not search : Prof Alessio (Serafozzi) Serafini


In [41]:
Final_Data = []
for i in final_prof_site:
    Final_Data.append(profile_scraper(i))

In [56]:
Final_Data

[['Samson Abramsky',
  ['16110', '3929'],
  ['58', '32'],
  ['126', '76'],
  [['Domain theory',
    'Samson Abramsky, Achim Jung',
    '1994/1/1',
    ' This text is based on the chapter Domain Theory in the Handbook for Logic in Computer Science, volume 3, edited by S. Abramsky, Dov M. Gabbay, and TS E. Maibaum, published by Clarendon Press, Oxford in 1994.',
    '1646'],
   ['A categorical semantics of quantum protocols',
    'Samson Abramsky, Bob Coecke',
    '2004',
    ' The particular examples we shall use to illustrate our approach will be teleportation (Benett et al., 1993), logic-gate teleportation (Gottesman and Chuang,1999), and entanglement swapping (Zukowski et al., 1993).',
    '877'],
   ['Computational Interpretations of Linear logic',
    'Samson Abramsky',
    '1993',
    ' In the case of Intuitionistic linear logic, this leads to a refinement of the lambda calculus, giving finer control over order of evaluation and storage allocation, while maintaining the logical co

In [52]:
Final_Data1 = Final_Data

In [55]:
Final_Data1

[['Samson Abramsky',
  ['16110', '3929'],
  ['58', '32'],
  ['126', '76'],
  [['Domain theory',
    'Samson Abramsky, Achim Jung',
    '1994/1/1',
    ' This text is based on the chapter Domain Theory in the Handbook for Logic in Computer Science, volume 3, edited by S. Abramsky, Dov M. Gabbay, and TS E. Maibaum, published by Clarendon Press, Oxford in 1994.',
    '1646'],
   ['A categorical semantics of quantum protocols',
    'Samson Abramsky, Bob Coecke',
    '2004',
    ' The particular examples we shall use to illustrate our approach will be teleportation (Benett et al., 1993), logic-gate teleportation (Gottesman and Chuang,1999), and entanglement swapping (Zukowski et al., 1993).',
    '877'],
   ['Computational Interpretations of Linear logic',
    'Samson Abramsky',
    '1993',
    ' In the case of Intuitionistic linear logic, this leads to a refinement of the lambda calculus, giving finer control over order of evaluation and storage allocation, while maintaining the logical co

In [54]:
Final_Data1 = summarize_abstracts(Final_Data)


ZeroDivisionError: division by zero

In [16]:
Final_Data1 = Final_Data
i = 0
j = 0
k = 0
for i in range(len(Final_Data1)):
    for k in range(10):
        if Final_Data[i][4][k][3] != "":
            Final_Data1[i][4][k][3] = summarize(Final_Data[i][4][k][3])
            print(i, j, k, len(Final_Data1[i][4][k][3]))
    for k in range(5):
        if Final_Data[i][5][k][3] != "":
            Final_Data1[i][5][k][3] = summarize(Final_Data[i][5][k][3])
            print(i, j, k, len(Final_Data1[i][5][k][3]))


1 0 1 0
4 0 3 0
7 0 4 0
7 0 5 0
8 0 3 0
10 0 2 0
10 0 4 0
15 0 1 0
15 0 2 0
16 0 2 0
17 0 0 0
17 0 1 0
23 0 4 0
24 0 8 0
26 0 2 0
26 0 9 0
27 0 1 0
28 0 9 0
28 0 0 0


In [37]:
Final_Data1 = Final_Data

In [30]:
article_scrape("https://scholar.google.com/citations?view_op=view_citation&hl=en&user=L2vmD3MAAAAJ&citation_for_view=L2vmD3MAAAAJ:C33y2ycGS3YC")

['Domain theory',
 'Samson Abramsky, Achim Jung',
 '1994/1/1',
 'This text is based on the chapter Domain Theory in the Handbook for Logic in Computer Science, volume 3, edited by S. Abramsky, Dov M. Gabbay, and TS E. Maibaum, published by Clarendon Press, Oxford in 1994. While the numbering of all theorems and definitions has been kept the same, we have included comments and corrections which we have received over the years. For ease of reading, small typographical errors have simply been corrected. Where we felt the original text gave a misleading impression, we have included additional explanations, clearly marked as such.',
 '1646']

In [57]:
df = pd.DataFrame(Final_Data, columns = ["Name", "[Total Citations, Citiations in the last 5 years]", "[H-index, H-index for last 5 years]", "[i10-index, i10-index for last 5 years]", "Most cited articles (in the form: [Title, Author, Publishing Date, Description, Citations]", "Most recent articles (in the form: [Title, Author, Publishing Date, Description, Citations]"])

In [58]:
df

,Name,"[Total Citations, Citiations in the last 5 years]","[H-index, H-index for last 5 years]","[i10-index, i10-index for last 5 years]","Most cited articles (in the form: [Title, Author, Publishing Date, Description, Citations]","Most recent articles (in the form: [Title, Author, Publishing Date, Description, Citations]"
0,Samson Abramsky,"[16110, 3929]","[58, 32]","[126, 76]","[[Domain theory, Samson Abramsky, Achim Jung, ...","[[Dagstuhl Reports, Vol. 12, Issue 2 ISSN 2192..."
1,Peter B. Barker,"[28611, 11049]","[89, 55]","[284, 209]",[[Computer-optimized decoupling scheme for wid...,[[Increased NAFLD Independent of BMI in South ...
2,Sougato Bose,"[17301, 6202]","[62, 41]","[148, 106]",[[Quantum communication through an unmodulated...,[[Gravitational Optomechanics: Photon-Matter E...
3,Daniel Brown,"[73123, 64172]","[93, 82]","[245, 193]",[[Observation of gravitational waves from a bi...,[[Model-based cross-correlation search for gra...
4,Cheng Guanghua,"[2291, 1340]","[24, 19]","[62, 49]",[[Micro Fabry-Perot interferometers in silica ...,[[Nano-spheroid formation on YAG surfaces indu...
5,Toby S. Cubitt,"[3044, 1824]","[26, 23]","[39, 34]","[[Assessing non-Markovian quantum dynamics, Mi...",[[A mathematical framework for quantum Hamilto...
6,Neil Curson,"[2851, 936]","[28, 15]","[50, 23]",[[Atomically precise placement of single dopan...,[[Non-destructive X-ray imaging of patterned d...
7,Agapi Emmanouilidou,"[960, 522]","[18, 14]","[34, 19]",[[Recoil collisions as a portal to field-assis...,[[Momentum scalar triple product as a measure ...
8,Andrew Fisher,"[9999, 6156]","[51, 38]","[123, 97]","[[Changing demographics, epidemiology, and sur...",[[Immune processes in the pathogenesis of chro...
9,Andy J. Green,"[18353, 8625]","[64, 43]","[254, 180]",[[New perspectives for estimating body conditi...,[[Comparison of taxonomic and size-based appro...


In [38]:
Final_Data1

[['Samson Abramsky',
  ['16110', '3929'],
  ['58', '32'],
  ['126', '76'],
  [['Domain theory', 'Samson Abramsky, Achim Jung', '1994/1/1', '', '1646'],
   ['A categorical semantics of quantum protocols',
    'Samson Abramsky, Bob Coecke',
    '2004',
    '',
    '877'],
   ['Computational Interpretations of Linear logic',
    'Samson Abramsky',
    '1993',
    '',
    '727'],
   ['Domain Theory in Logical Form', 'Samson Abramsky', '1991', '', '688'],
   ['The Lazy Lambda Calculus', 'Samson Abramsky', '1990', '', '680'],
   ['Full Abstraction for PCF',
    'Samson Abramsky, Radha Jagadeesan, Pasquale Malacaria',
    '2000',
    '',
    '626'],
   ['Games and full completeness for multiplicative linear logic',
    'Samson Abramsky, Radha Jagadeesan',
    '1994/6',
    '',
    '481'],
   ['The sheaf-theoretic structure of non-locality and contextuality',
    'Samson Abramsky, Adam Brandenburger',
    '2011/6/21',
    '',
    '467'],
   ['Abstract interpretation of declarative languages',


In [19]:
df

,Name,"[Total Citations, Citiations in the last 5 years]","[H-index, H-index for last 5 years]","[i10-index, i10-index for last 5 years]","Most cited articles (in the form: [Title, Author, Publishing Date, Description, Citations]","Most recent articles (in the form: [Title, Author, Publishing Date, Description, Citations]"


In [48]:
C = 'This text is based on the chapter Domain Theory in the Handbook for Logic in Computer Science, volume 3, edited by S. Abramsky, Dov M. Gabbay, and TS E. Maibaum, published by Clarendon Press, Oxford in 1994. While the numbering of all theorems and definitions has been kept the same, we have included comments and corrections which we have received over the years. For ease of reading, small typographical errors have simply been corrected. Where we felt the original text gave a misleading impression, we have included additional explanations, clearly marked as such.'


In [49]:
summarize(C)

' This text is based on the chapter Domain Theory in the Handbook for Logic in Computer Science, volume 3, edited by S. Abramsky, Dov M. Gabbay, and TS E. Maibaum, published by Clarendon Press, Oxford in 1994.'